<a href="https://colab.research.google.com/github/dipan95/Coursera-Machine-Learning-/blob/master/MediaValue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor # import the random forest model
from sklearn import  preprocessing # used for label encoding and imputing NaNs

import datetime as dt # we will need this to convert the date to a number of days since some point

from sklearn.tree import export_graphviz
import pydotplus
import six
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

import matplotlib.pyplot as plt
%matplotlib inline


In [79]:
path_train = '/content/drive/My Drive/Data.csv'

In [97]:
df=pd.read_csv(path_train,parse_dates=['posttime'])
df.head()

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,mediavalue,videoviews,impressions,engagement
0,59087769,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-10 09:51:00,35600.0,3.751751,480.0,NaN,18.0
1,72783016,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-05-20 16:34:00,35400.0,2.790141,355.0,NaN,22.0
2,58838506,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-07 15:51:00,35600.0,0.979764,213.0,NaN,11.0
3,63598598,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-02-26 13:18:00,35500.0,0.603062,1675.0,NaN,35.0
4,70097609,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-06 23:13:00,544.0,3.930096,149.0,332.0,2.0


In [177]:
df.shape

(603634, 14)

In [178]:
df.tail()

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,mediavalue,videoviews,impressions,engagement
603634,63970878,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-27 19:07:00,95700.0,7.80668,5794.0,NaN,223.0
603635,65875255,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-03-05 14:21:00,96300.0,0.00000,12840.0,NaN,388.0
603636,63040326,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-20 18:24:00,95400.0,0.00000,13681.0,NaN,525.0
603637,62529906,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-13 14:36:00,94300.0,0.00000,18501.0,NaN,612.0
603638,62377379,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-01-22 20:52:00,92300.0,0.00000,3539.0,NaN,288.0


In [179]:
df.describe()

,id,isbranded,pssid,followers,mediavalue,videoviews,impressions,engagement
count,6.036340e+05,603634.000000,603634.000000,6.036340e+05,6.036340e+05,1.381010e+05,5.875300e+05,6.036340e+05
mean,6.592363e+07,0.035034,1.020569,3.682473e+06,6.043297e+02,1.580049e+05,1.805377e+05,5.960308e+03
std,4.089130e+06,0.183867,0.141936,9.331514e+06,8.111492e+03,6.146424e+05,7.954558e+05,4.359428e+04
min,5.823608e+07,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.253687e+07,0.000000,1.000000,1.812350e+05,0.000000e+00,1.726000e+03,4.931000e+03,2.100000e+01
50%,6.594106e+07,0.000000,1.000000,6.759315e+05,0.000000e+00,1.899700e+04,4.123200e+04,1.040000e+02
75%,6.968584e+07,0.000000,1.000000,2.263974e+06,9.970885e+00,7.051200e+04,8.545800e+04,8.080000e+02
max,7.466514e+07,1.000000,2.000000,1.569743e+08,1.349303e+06,4.141426e+07,9.203170e+07,7.745832e+06


In [180]:
df.isbranded.unique()

array([0, 1])

In [181]:
df = df[df['pssid'].notna()]

In [182]:
df_train=df.loc[df['mediavalue'] != 0]
df_test=df.loc[df['mediavalue'] == 0]

In [183]:
df_train.shape
df_train

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,mediavalue,videoviews,impressions,engagement
0,59087769,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-10 09:51:00,35600.0,3.751751,480.0,NaN,18.0
1,72783016,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-05-20 16:34:00,35400.0,2.790141,355.0,NaN,22.0
2,58838506,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-07 15:51:00,35600.0,0.979764,213.0,NaN,11.0
3,63598598,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-02-26 13:18:00,35500.0,0.603062,1675.0,NaN,35.0
4,70097609,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-06 23:13:00,544.0,3.930096,149.0,332.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603630,62369845,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-13 18:37:00,94300.0,42.772590,9534.0,NaN,319.0
603631,63970877,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-27 19:10:00,95700.0,30.288500,5525.0,NaN,230.0
603632,65875256,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-03-05 14:38:00,96300.0,29.729401,20679.0,NaN,666.0
603633,63040253,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-20 21:34:00,95400.0,17.634300,32818.0,NaN,789.0


In [184]:
df_train=df_train.reset_index(drop=True)
df_train

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,mediavalue,videoviews,impressions,engagement
0,59087769,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-10 09:51:00,35600.0,3.751751,480.0,NaN,18.0
1,72783016,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-05-20 16:34:00,35400.0,2.790141,355.0,NaN,22.0
2,58838506,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-01-07 15:51:00,35600.0,0.979764,213.0,NaN,11.0
3,63598598,YouTube,Video,1080football (ytube),SPORF,Media,0,1.0,2020-02-26 13:18:00,35500.0,0.603062,1675.0,NaN,35.0
4,70097609,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-06 23:13:00,544.0,3.930096,149.0,332.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197663,62369845,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-13 18:37:00,94300.0,42.772590,9534.0,NaN,319.0
197664,63970877,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-27 19:10:00,95700.0,30.288500,5525.0,NaN,230.0
197665,65875256,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-03-05 14:38:00,96300.0,29.729401,20679.0,NaN,666.0
197666,63040253,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-20 21:34:00,95400.0,17.634300,32818.0,NaN,789.0


In [185]:
df_test=df_test.reset_index(drop=True)
df_test

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,mediavalue,videoviews,impressions,engagement
0,72207570,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-29 15:46:00,542.0,0.0,3.0,6.0,0.0
1,72117480,Facebook,Link,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-28 21:07:00,542.0,0.0,NaN,2.0,0.0
2,72117497,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-28 17:30:00,542.0,0.0,76.0,169.0,1.0
3,71314947,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-20 17:30:00,542.0,0.0,3.0,6.0,0.0
4,71314954,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-20 16:56:00,542.0,0.0,3.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405961,69908357,Twitter,Video,Yankees (twit),Yankees,Team,0,1.0,2020-05-04 15:03:00,3458646.0,0.0,28093.0,62428.0,1315.0
405962,65875255,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-03-05 14:21:00,96300.0,0.0,12840.0,NaN,388.0
405963,63040326,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-20 18:24:00,95400.0,0.0,13681.0,NaN,525.0
405964,62529906,YouTube,Video,yankees (ytube),Yankees,Team,0,1.0,2020-02-13 14:36:00,94300.0,0.0,18501.0,NaN,612.0


In [186]:
y_train_mediavalue=df_train['mediavalue']

In [187]:
df_train = df_train.drop(['mediavalue'], axis=1)
df_test=df_test.drop(['mediavalue'],axis=1)

In [188]:
df_test.head()

,id,platform,postType,Channel,choName,chotName,isbranded,pssid,posttime,followers,videoviews,impressions,engagement
0,72207570,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-29 15:46:00,542.0,3.0,6.0,0.0
1,72117480,Facebook,Link,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-28 21:07:00,542.0,NaN,2.0,0.0
2,72117497,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-28 17:30:00,542.0,76.0,169.0,1.0
3,71314947,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-20 17:30:00,542.0,3.0,6.0,0.0
4,71314954,Facebook,Video,1159151984219605 (fbook),76ers GC,Team,0,1.0,2020-05-20 16:56:00,542.0,3.0,6.0,0.0


In [189]:
df_train["trainOrTest"] = "train"
df_test["trainOrTest"] = "test"
df_all = pd.concat([df_train, df_test])

In [190]:
df_all.shape

(603634, 14)

In [191]:
# Convert the Post time into a number (of days since some point)
fromDate = min(df_all['posttime'])
df_all['posttime_new'] = (df_all['posttime'] - fromDate).dt.days.astype(int)
print(df_all[['posttime_new','posttime']].head())
df_all.drop('posttime', axis = 1, inplace = True)

   posttime_new            posttime
0             9 2020-01-10 09:51:00
1           140 2020-05-20 16:34:00
2             6 2020-01-07 15:51:00
3            56 2020-02-26 13:18:00
4           126 2020-05-06 23:13:00


In [192]:
df.choName.unique()

array(['SPORF', '76ers GC', '20 Minutes', '247Sports', 'Gerard Piqué',
       '433', 'ESPN Australia', '7AFL', 'ABC Grandstand',
       'Auto Bild Motorsport', 'Audi Team Abt Sportsline',
       'Adelaide Crows', 'Wide World of Sports', 'AFC Bournemouth', 'AFL',
       'AFL Footy Show', 'fox footy', 'Alaves', 'Amiens SC',
       'Andrea Kaiser', 'Angels', 'Angers SCO', 'Antoine Griezmann',
       'Liverpool', 'Aral', 'Arsenal', 'ART Grand Prix', 'As',
       'AS Monaco', 'AS Saint-Etienne', 'Astros', 'Athletic Club Bilbao',
       'Athletics', 'Atlanta United FC', 'Atletico Madrid', 'ATS Wheels',
       'audi.dtm', 'Audi Sport Seyffarth R8 LMS Cup',
       'Audi30 / AudiSport30', 'Autodrom Nazionale Monza', 'Automoto',
       'Auto Motor und Sport', 'Autosport', 'Aston Villa',
       'AZTECA DEPORTES', 'Gareth Bale', 'F.C. Barcelona',
       'Barstool Sports', 'BBC SPORT', 'BBC MOTD', 'beIN Sports',
       'Karim Benzema', 'BFMTV', 'Brighton & Hove Albion', 'BILD',
       'Bild Sport',

In [193]:
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_all[c].values)) 
        df_all[c] = lbl.transform(list(df_all[c].values))

In [194]:
df_all.drop('id', axis = 1, inplace = True)

In [195]:
df_all

,platform,postType,Channel,choName,chotName,isbranded,pssid,followers,videoviews,impressions,engagement,trainOrTest,posttime_new
0,3,4,0,258,3,0,1.0,35600.0,480.0,NaN,18.0,1,9
1,3,4,0,258,3,0,1.0,35400.0,355.0,NaN,22.0,1,140
2,3,4,0,258,3,0,1.0,35600.0,213.0,NaN,11.0,1,6
3,3,4,0,258,3,0,1.0,35500.0,1675.0,NaN,35.0,1,56
4,0,4,1,3,4,0,1.0,544.0,149.0,332.0,2.0,1,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
405961,2,4,539,326,4,0,1.0,3458646.0,28093.0,62428.0,1315.0,0,124
405962,3,4,1185,326,4,0,1.0,96300.0,12840.0,NaN,388.0,0,64
405963,3,4,1185,326,4,0,1.0,95400.0,13681.0,NaN,525.0,0,50
405964,3,4,1185,326,4,0,1.0,94300.0,18501.0,NaN,612.0,0,43


In [196]:
#from numpy import isnan
#from sklearn.impute import KNNImputer
#import scipy.sparse as sparse

In [197]:
#Null values in IMPRESSIONS
#fill nan value of 'condition' with KNNImputer
#X=df_all[["impressions"]]
#imputer = KNNImputer(n_neighbors=2)
#imputer.fit_transform(X)
#array2d=imputer.fit_transform(X)
#result = array2d.flatten()
#resultf=np.around(result) 
#print(resultf)

In [198]:
#Dm = pd.DataFrame(resultf.reshape(len(resultf),1),columns=['impressions_edited'])
#print (Dm)

In [199]:
#frames = [df_all, Dm]
#df_all= pd.concat(frames, axis=1, sort=False)

In [200]:
df_all=df_all.fillna(np.around(df_all.mean()))

In [201]:
df_all.isnull().any()

platform        False
postType        False
Channel         False
choName         False
chotName        False
isbranded       False
pssid           False
followers       False
videoviews      False
impressions     False
engagement      False
trainOrTest     False
posttime_new    False
dtype: bool

In [202]:
df_all.head()

,platform,postType,Channel,choName,chotName,isbranded,pssid,followers,videoviews,impressions,engagement,trainOrTest,posttime_new
0,3,4,0,258,3,0,1.0,35600.0,480.0,180538.0,18.0,1,9
1,3,4,0,258,3,0,1.0,35400.0,355.0,180538.0,22.0,1,140
2,3,4,0,258,3,0,1.0,35600.0,213.0,180538.0,11.0,1,6
3,3,4,0,258,3,0,1.0,35500.0,1675.0,180538.0,35.0,1,56
4,0,4,1,3,4,0,1.0,544.0,149.0,332.0,2.0,1,126


In [203]:
df_all.isnull().sum()

platform        0
postType        0
Channel         0
choName         0
chotName        0
isbranded       0
pssid           0
followers       0
videoviews      0
impressions     0
engagement      0
trainOrTest     0
posttime_new    0
dtype: int64

In [204]:
# Prepare separate train and test datasets
idx_train = df_all['trainOrTest'] == 1
idx_test = df_all['trainOrTest'] == 0

x_train = df_all[idx_train]
x_test = df_all[idx_test]

In [205]:
df_all.drop('trainOrTest', axis = 1,inplace=True)
x_train.drop('trainOrTest', axis = 1,inplace=True)
x_test.drop('trainOrTest', axis = 1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [207]:
x_test

,platform,postType,Channel,choName,chotName,isbranded,pssid,followers,videoviews,impressions,engagement,posttime_new
0,0,4,1,3,4,0,1.0,542.0,3.0,6.0,0.0,149
1,0,1,1,3,4,0,1.0,542.0,158005.0,2.0,0.0,148
2,0,4,1,3,4,0,1.0,542.0,76.0,169.0,1.0,148
3,0,4,1,3,4,0,1.0,542.0,3.0,6.0,0.0,140
4,0,4,1,3,4,0,1.0,542.0,3.0,6.0,0.0,140
...,...,...,...,...,...,...,...,...,...,...,...,...
405961,2,4,539,326,4,0,1.0,3458646.0,28093.0,62428.0,1315.0,124
405962,3,4,1185,326,4,0,1.0,96300.0,12840.0,180538.0,388.0,64
405963,3,4,1185,326,4,0,1.0,95400.0,13681.0,180538.0,525.0,50
405964,3,4,1185,326,4,0,1.0,94300.0,18501.0,180538.0,612.0,43


In [209]:
x_test.describe()

,platform,postType,Channel,choName,chotName,isbranded,pssid,followers,videoviews,impressions,engagement,posttime_new
count,405966.000000,405966.000000,405966.000000,405966.000000,405966.000000,405966.0,405966.000000,4.059660e+05,4.059660e+05,4.059660e+05,4.059660e+05,405966.000000
mean,1.632193,2.596230,473.749644,169.873332,3.548120,0.0,1.011279,3.303370e+06,1.564428e+05,1.333312e+05,4.362327e+03,80.718868
std,0.757542,0.898689,316.364762,91.688345,0.647447,0.0,0.105603,8.145557e+06,2.084124e+05,5.944527e+05,3.678633e+04,42.663471
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,2.000000,2.000000,245.000000,85.000000,3.000000,0.0,1.000000,2.175370e+05,1.580050e+05,4.934000e+03,1.400000e+01,46.000000
50%,2.000000,3.000000,394.000000,173.000000,4.000000,0.0,1.000000,7.402960e+05,1.580050e+05,4.326400e+04,6.900000e+01,82.000000
75%,2.000000,3.000000,717.000000,242.000000,4.000000,0.0,1.000000,2.314536e+06,1.580050e+05,8.346900e+04,5.130000e+02,119.000000
max,3.000000,4.000000,1185.000000,335.000000,4.000000,0.0,2.000000,1.569743e+08,4.141426e+07,9.203170e+07,7.745832e+06,151.000000
